In [16]:
from pyomo.environ import *
from pyomo.gdp import *

model = ConcreteModel()

# Parameters
model.C_ch = Param(within=NonNegativeReals, default=10)
model.C_m = Param(domain=NonNegativeReals, default=15)
model.Prod_ch = Param(domain=NonNegativeReals, default=2)
model.Prod_m = Param(domain=NonNegativeReals, default=3)
model.CAP = Param(domain=NonNegativeIntegers, default=100)
model.O_min = Param(domain=NonNegativeIntegers, default=10)
model.D_l = Param(domain=NonNegativeReals, default=20)
model.D_c = Param(domain=NonNegativeReals, default=30)

# GDP parameters
model.Cextra = Param(within=NonNegativeReals, default=200)
model.limit_cap = Param(within=NonNegativeIntegers, default=20)

# Variables
model.X = Var(domain=NonNegativeIntegers)
model.Y = Var(domain=NonNegativeIntegers)

# Constraints
model.res1 = Constraint(expr=model.X + model.Y >= model.O_min)
model.res2 = Constraint(expr=model.Y >= 2*model.X)
model.res3 = Constraint(expr=model.X + model.Y <= model.CAP)
model.res4 = Constraint(expr=model.Prod_ch * model.X >= model.D_l)
model.res5 = Constraint(expr=model.Prod_m * model.Y >= model.D_c)

model.extra_cost_applies = Disjunct()
model.extra_cost_applies.rule = Constraint(expr=model.X + model.Y >= model.limit_cap)
model.extra_cost_not_applies = Disjunct()
model.extra_cost_not_applies.rule = Constraint(expr=model.X + model.Y <= model.limit_cap)
model.disj = Disjunction(expr=[model.extra_cost_applies, model.extra_cost_not_applies])

# Objective
model.obj = Objective(expr=model.C_ch * model.X + model.C_m * model.Y, sense=minimize)

In [17]:
xfrm = TransformationFactory('gdp.bigm')
xfrm.apply_to(model, bigM=1000)
solver = SolverFactory('glpk')
status = solver.solve(model)

# Results
print("X value:", model.X.value)
print("Y value:", model.Y.value)
print('Is extra cost applied?:', model.extra_cost_applies.indicator_var.value)
print("Obj value:", value(model.obj))

X value: 10.0
Y value: 20.0
Is extra cost applied?: True
Obj value: 400.0
